In [61]:
from sklearn.cross_validation import train_test_split
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [62]:
import numpy as np
with open('pos.txt','r') as infile:
    pos_ = infile.readlines()
with open('neg.txt','r')  as infile:
    neg_ = infile.readlines()
with open('target.txt','r') as infile:
    tar = infile.readlines()

y =  np.concatenate((np.ones(len(pos_)),np.zeros(len(neg_))))
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos_, neg_)), y ,test_size = 0.2)

def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        label = '%s_%s'%(label_type, i )
        labelized.append(LabeledSentence(v, [label]))
    return labelized
x_train = labelizeReviews(x_train, 'TRAIN')
x_test = labelizeReviews(x_test, 'TEST')
tar = labelizeReviews(tar, 'TAR')


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  app.launch_new_instance()


In [63]:
import random
size = 400
model_dm = gensim.models.Doc2Vec(min_count = 1, window =10, size = size, sample = 0.001, negative =5, worker =3)
model_dbow = gensim.models.Doc2Vec(min_count = 1, window =10, size = size, sample = 0.001, negative =5, dm = 0, worker =3)

D:\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [64]:
model_dm.build_vocab(np.concatenate(((x_train, x_test, tar)))
model_dbow.build_vocab(np.concatenate(((x_train, x_test, tar)))
#model_dbow.build_vocab(np.concatenate((x_train, x_test, tar)))
all_train_reviews = np.concatenate(((x_train, tar))

for epoch in range(10):
    perm = np.random.permutation((x_train, tar))
    model_dm.train(all_train_reviews[perm])
    model_dbow.train(all_train_reviews[perm])


SyntaxError: invalid syntax (<ipython-input-64-b5c86d63c830>, line 2)

In [24]:
def getvecs(model,corpus, size):
    vecs = [np.array(model[z.labels[0]]).reshape((1, size)) for z in corpus]
    return np.concatenate(vecs)
train_vecs_dm = getvecs(model_dm, x_train, size)
train_vecs_dbow = getvecs(model_dbow, x_train, size)
train_vecs = np.hstack((train_vecs_dm, train_vecs_dbow))
x_test = np.array(x_test)
for epoch in range(10):
    perm = np.random.permutation(x_test.shape[0])
    model_dm.train(x_test[[perm]])
    model_dbow.train(x_test[perm])

    
test_vecs_dm = getvecs(model_dm, x_test, size)
test_vecs_dbow = getvecs(model_dbow, x_test, size)
test_vecs = np.hstack((test_vecs_dm, test_vecs_dbow))

AttributeError: 'LabeledSentence' object has no attribute 'labels'

In [ ]:
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss = 'log',penalty = 'll')
lr.fit(train_vecs, y_train)
print('Test Accuracy: %2f' %lr.score(test_vecs, y_test))

In [ ]:
b